In [ ]:
import os
from pathlib import Path
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import contextily as cx
import geopandas

In [ ]:
# List of stations to loop through
station = ['BBY', 'CAT', 'Kibbe', 'NBB', 'SOD', 'Tacoma']
station_coords = {'BBY': [38.3183, -123.0729], 'CAT': [33.4462, -118.4828], 'Kibbe': [39.2208, -121.4822],
                      'NBB': [39.3964, -121.1438], 'SOD': [34.1148, -117.0875], 'Tacoma': [47.1338, -122.2569]}

In [ ]:
filename = './data/USBOD/USBOD_20250214_0000.txt'

In [ ]:
df_sounding = pd.read_csv(filename, names=['Year', 'Month', 'Day', 'Hour', 'Minute',
                                                                'Second', 'Pressure (hPa)', 'Temperature (C)',
                                                                'Dew Point (C)', 'Wind Direction (degrees)',
                                                                'Wind Speed (m/s)', 'Height (m)', 'Latitude',
                                                                'Longitude'], delim_whitespace=True)

In [ ]:
extents = [-123.5, -121, 37.5, 39.25]
fig, ax = plt.subplots(figsize=(10, 8))
ax.axis(extents)

df_sounding_subset = df_sounding[['Temperature (C)', 'Latitude', 'Longitude']]

gdf_sounding_subset = geopandas.GeoDataFrame(
                    df_sounding_subset, geometry=geopandas.points_from_xy(df_sounding_subset['Longitude'],
                                                                          df_sounding_subset['Latitude']))

gdf_sounding_subset = gdf_sounding_subset.set_crs(epsg=4326)

cx.add_basemap(ax, source=cx.providers.OpenStreetMap.Mapnik, crs=gdf_sounding_subset.crs, zoom=9)

gdf_sounding_subset.plot(kind='scatter', x='Longitude', y='Latitude', ax=ax, color='grey')

station_coord = station_coords['BBY']

df_coords = pd.DataFrame(columns=['Latitude', 'Longitude'])

df_coords.at[0, 'Latitude'] = station_coord[0]
df_coords.at[0, 'Longitude'] = station_coord[1]

gdf_coords = geopandas.GeoDataFrame(df_coords, geometry=geopandas.points_from_xy(
    df_coords['Longitude'], df_coords['Latitude']))

gdf_coords.plot(ax=ax, color='red')

df_sounding_flipped = df_sounding.iloc[::-1]
df_sounding_flipped = df_sounding_flipped.reset_index(drop=True)

# Get temperature and height from flipped dataframe to search from top down
temps_flipped = np.array(df_sounding_flipped['Temperature (C)'])
heights_flipped = np.array(df_sounding_flipped['Height (m)'])

# Get the highest level where sign change occurs
temp_cross = np.where(np.sign(temps_flipped[:-1]) != np.sign(temps_flipped[1:]))[0]

date_to_process = datetime.utcnow() - timedelta(days=40)
if (date_to_process.month == 10) or (date_to_process.month == 11) or (date_to_process.month == 12):
        water_year = str(date_to_process.year) + '-' + str(date_to_process.year + 1)
else:
        water_year = str(date_to_process.year - 1) + '-' + str(date_to_process.year)

 # If a sign change occurs, that becomes the freezing level
if len(temp_cross) > 0:
                    # Assign value to temp array
                    sounding_freeze_obs = heights_flipped[temp_cross][0]

                    # Create geopandas dataframe
                    df_0c = df_sounding_flipped.iloc[temp_cross[0]-1:temp_cross[0], :]
                    gdf_0c = geopandas.GeoDataFrame(df_0c, geometry=geopandas.points_from_xy(df_0c['Longitude'],
                                                                                             df_0c['Latitude']))

                    # Plot the 0C location as a blue dot
                    gdf_0c.plot(kind='scatter', x='Longitude', y='Latitude', ax=ax, color='blue')

                # If no sign change is found - freezing level is NaN
else:
                    sounding_freeze_obs = np.nan

                # Add title
ax.set_title(str(filename), fontsize=14)
plt.tight_layout()
plt.show()